# Создание RAG для диалоговых систем

## Загрузка необходимых библиотек

In [ ]:
from config import WIKI_NOTES_PATH, NUMBER_DOC_FROM_WIKI, CHUNK_SIZE, CHUNK_OVERLAP, CHROMA_PATH, EMBEDDING_MODEL, COLLECTION_NAME
from config import SIMILARITY_THRESHOLD, QUESTIONS_DIALOG, ANSWERS_DIALOG, NUMBER_DOCS_FOR_CONTEXT, LLM_MODEL

import torch
import transformers
from langchain_huggingface import HuggingFaceEmbeddings
#from langchain_chroma import Chroma
#from langchain_core.documents import Document
from langchain.vectorstores import Chroma
from langchain_core.documents.base import Document
from langchain_core.prompts import ChatPromptTemplate
from transformers import AutoTokenizer, AutoModelForCausalLM, AwqConfig

import shutil
from scipy.spatial.distance import euclidean

from googlesearch import search
import requests
from bs4 import BeautifulSoup
import urllib
import re
import json
from tqdm import tqdm

from db_class import ChromaDB
from func import split_text

data_path = ''
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Создание векторной БД
База знаний состоит из первых абзацев русскоязычных статей из википедии про различных людей.

In [ ]:
db_obj = ChromaDB(data_path+CHROMA_PATH, EMBEDDING_MODEL)
db_obj.create_chromadb_from_filedata(data_path + WIKI_NOTES_PATH, NUMBER_DOC_FROM_WIKI, COLLECTION_NAME, CHUNK_SIZE, CHUNK_OVERLAP)

start save ...
save 100(140 chunks) to bd success!


In [ ]:
# просто выведем для проверки 2 первых документа из бд
db_obj.get_bd_documents(COLLECTION_NAME, 2)

{'ids': ['042f8b5e-22d3-4920-838f-89065af26200',
  '04aee381-3917-4b8f-a5d7-ea59214bf62b'],
 'embeddings': None,
 'metadatas': [{'start_index': 0}, {'start_index': 0}],
 'documents': ['Берна́рдо Бертолу́ччи (; 16 марта 1941, Парма, Италия — 26 ноября 2018, Рим) — итальянский кинорежиссёр, драматург и поэт.В 1960-е годы выступал как последователь Годара и Пазолини, увлекался коммунизмом и фрейдизмом, тонко переплетая в своих фильмах социальное с интимным.Во многих лентах обращался к табуированным формам человеческой сексуальности — инцесту, триолизму, гомосексуальности.Постоянно сотрудничал с оператором Витторио Стораро, прозванным итальянскими критиками «богом светотени».После триумфального успеха фильмов «Конформист» (1970) и «Последнее танго в Париже» (1972) работал за пределами Италии, в том числе на буддийском Востоке, называя себя «скептическим буддистом-любителем».Обладатель премии «Оскар» за постановку и написание сценария к эпическому байопику «Последний император» (1987).',
  

## Интернет поиск
В случае, когда не окажется в бд информации по воспросу - будем обращаться к интернету

In [ ]:
def get_link_text(link):
  headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6367.60 Safari/537.3'
  }
  params = {"hl": "ru"}
  response = requests.get(link, params=params, headers=headers)
  soup = BeautifulSoup(response.text, "html.parser")
  text = re.sub(r'[\n]+', '\n', soup.get_text()).replace('\xa0', ' ')
  return text


def internet_search(query, k) -> list:
  docs = []
  for url in search(query, stop=k):
    try:
      text = get_link_text(urllib.parse.unquote(url))
      docs.append(text)
    except:
      pass
  return docs

In [ ]:
#internet_search('Кто был президентом США, подписавшим Прокламацию об освобождении рабов?', 3)

## Поддержка диалогов и построение RAG системы

**Загрузим необходимые модели, БД и сами вопросы, на которые нужно будет сгенерировать ответ**

In [ ]:
# токенизатор
tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL)
#model = AutoModelForCausalLM.from_pretrained("hugging-quants/Meta-Llama-3.1-8B-Instruct-AWQ-INT4")

# квантизованная модель LLM
quantization_config = AwqConfig(bits=4, fuse_max_seq_len=2048, do_fuse=True)
model = AutoModelForCausalLM.from_pretrained(
    LLM_MODEL,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    device_map="auto",
    quantization_config=quantization_config
)
model.to(device)

# модель для эмбеддингов
model_kwargs = {'device': device}
embeddings_hf = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL,
    model_kwargs=model_kwargs
)

# Подгрузка БД Chroma
db = Chroma(persist_directory=data_path + CHROMA_PATH, embedding_function=embeddings_hf, collection_name=COLLECTION_NAME)

# Чтение списка вопросов, на которые необходимо ответить
with open(data_path + QUESTIONS_DIALOG, 'r') as f:
    dialog_questions = f.read().split('\n\n')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/295 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

**Функции, необходимые для построения RAG системы**

In [ ]:
def get_query_by_history(dialog_history, curr_query, tokenizer, model, device) -> str:
  '''
    Функция для получения текста запроса по истории диалога

    Аргументы:
      dialog_history - список из кортежей, в которых на первом месте - кто говорит (пользователь/ИИ), на втором - сам текст
      curr_query - текущий запрос, который нужно переформулировать при наличии истории диалога
      tokenizer -токенизатор
      model - модель LLM

    Возвращаемое значение:
      Текст, содержащий уточняющий вопрос, сгенерированный LLM
  '''
  # если нет еще истории - текущий переформулированный вопрос- это сам исходный вопрос
  if not len(dialog_history):
    return curr_query
  # иначе составляем текст с историей диалога
  text_history = ''
  for role, value in dialog_history:
    text_history += f'{role}: {dialog_history}\n'
  # формируем промпт для модели
  system_message = f"""
      Ты ассистент, который читает запись разговора между искусственным интеллектом и пользователем.
      Имея историю разговора и новый запрос пользователя, переформулируй запрос с учетом истории так,
      чтобы он имел однозначный ответ. Замени все местоимения и общие слова на конкретные сущности.
      Не пиши ничего, кроме переформулированного запроса.
      История разговора: ```{text_history}```\n\n
      Запрос пользователя: ```{curr_query}```\n
  """

  messages = [
      {"role": "user", "content": system_message},
  ]
  inputs = tokenizer.apply_chat_template(
      messages,
      tokenize=True,
      add_generation_prompt=True,
      return_tensors="pt",
      return_dict=True,
  ).to(device)

  outputs = model.generate(**inputs, do_sample=True, temperature=0.1, max_new_tokens=512)[0]
  result = tokenizer.decode(outputs[inputs['input_ids'].shape[1]:], skip_special_tokens=True)
  return result


def get_relevant_docs(query, db) -> list:
  '''
    Функция для получения списка релевантных текстов для переданного вопроса

    Аргументы:
      query - запрос, к которому требуется подобрать наиболее близкие документы
      db - векторна БД с текстами

    Возвращаемое значение:
      Спсиок релевантных документов
  '''
  # сначала ищем подходящие тексты среди проверенных документов, хранимых в БД
  docs_scores = db.similarity_search_with_relevance_scores(query, k=NUMBER_DOCS_FOR_CONTEXT)
  documents = [doc.page_content for doc, sc in docs_scores if sc > SIMILARITY_THRESHOLD]
  # если документов с заданной похожестью в бд оказалось меньше положенного - идем в гугл
  if len(documents) < NUMBER_DOCS_FOR_CONTEXT:
    # в интернете нам нужно найти недостающее число документов
    num_doc_from_i = int(NUMBER_DOCS_FOR_CONTEXT - len(documents))
    ### todo для удобства поддержки залогировать запросы, на которые не достаточно информации в существующей БД
    print(f'Получаем информацию из интернета для вопроса "{query}"')
    documents_i = internet_search(query, num_doc_from_i)
    # разобъем наши документы из интернета на чанки, т.к. они могут быть весьма объемными
    chunks = split_text(
        [Document(page_content=doc) for doc in documents_i],
        500,
        100
    )
    # посчитаем эмбеддинги для запроса
    query_embedding = db._embedding_function.embed_query(query)
    for chunk in chunks:
      # посчитаем эмбеддинги для текущего чанка
      chunk_embedding = db._embedding_function.embed_query(chunk.page_content)
      # функцию подсчета скора пишем на основе той, что используется в нашей db
      score = db._select_relevance_score_fn(euclidean(chunk_embedding, query_emb)**2)
      if score > SIMILARITY_THRESHOLD:
          documents.append(chunk.page_content)
    '''
    #### todo перенести в одну БД, но с новой коллекцией
    # нам не нужно засорять бд непроверенными данными, поэтому будем сразу чистить ее
    client_i = chromadb.PersistentClient(path=data_path + INTERNET_CHROMA_PATH)
    try:
      client_i.delete_collection('famous_people')
    except:
      pass
    db_i = Chroma.from_documents(
      chunks, embeddings_hf, persist_directory=data_path + INTERNET_CHROMA_PATH, collection_name='famous_people'
    )
    # отбираем наиболее похожие тексты из интернета в недостающем кол-ве
    docs_scores = db_i.similarity_search_with_relevance_scores(dialog_question_curr, k=num_doc_from_i)
    documents_from_internet = [doc.page_content for doc, sc in docs_scores if sc > SIMILARITY_THRESHOLD]
    documents.extend(documents_from_internet)
    '''
  return documents


def get_answer_dialog(dialog_questions_str, db, tokenizer, model, device):
  '''
    Функция для генерации ответов на подряд идущие вопросы

    Аргументы:
      dialog_questions_str - текст с подряд идущими вопросами
      db - наша база знаний
      tokenizer - токенизатор
      model - модель LLM

    Возвращаемое значение:
      Список с ответами на вопросы
  '''
  dialog_questions = dialog_questions_str.split('\n')
  dialog_answers = []  # здесь будем хранить ответы на  вопросы

  dialog_history = []
  for que in dialog_questions:

    ########### УТОЧНЕНИЕ ВОПРОСА по истории диалога###########
    que_full = get_query_by_history(dialog_history, que, tokenizer, model, device)
    ########### ПОИСК КОНТЕКСТА ###########
    relevant_documents = get_relevant_docs(que_full, db)
    context = '\n'.join(relevant_documents)

    dialog_history.append(('Пользователь', que_full))
    ########### СОСТАВЛЕНИЕ ПРОМТА ###########
    system_message = f"""
        Ты полезный ассистент.
        Пожалуйста, дай ответ на запрос пользователя, используя только данную тебе информацию в контексте.
        Убедись, что твой ответ краток и точен и не содержит никакой другой информации.\n
        Контекст: ```{context}```\n\n
        Запрос: ```{que_full}```\n
    """
    messages = [
        {"role": "user", "content": system_message},
    ]

    ########### ГЕНЕРАЦИЯ ОТВЕТА ###########
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
        return_dict=True,
    ).to(device)
    with torch.no_grad():
      outputs = model.generate(**inputs, do_sample=True, temperature=0.03, max_new_tokens=300)[0]
    answers = tokenizer.decode(outputs[inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    result_dialog.append(answers)
    dialog_history.append(('ИИ', answers))

  return result_dialog

**Генерируем ответы на вопросы**

In [ ]:
generated_answers_dialog = []
for dialog_question in tqdm(dialog_questions):
  generated_answers_dialog.append(get_answer_dialog(dialog_question, db, tokenizer, model, device))


  0%|          | 0/1 [00:00<?, ?it/s]

Получаем информацию из интернета для вопроса "Кто основал компанию Tesla?"
Получаем информацию из интернета для вопроса "В каком году была основана компания Tesla?"


100%|██████████| 1/1 [01:51<00:00, 111.93s/it]


**Сохраним ответы на диске**

In [ ]:
with open(data_path + ANSWERS_DIALOG, 'w', encoding='utf8') as f:
    json.dump(generated_answers_dialog, f, ensure_ascii=False)

## Отображение результирующих ответов

In [ ]:
# загружаем сгенерированные ответы из файла
with open(data_path + ANSWERS_DIALOG, 'r', encoding='utf8') as f:
    generated_answers_dialog = json.load(f)
# отображаем ответы с вопросами
for i, ans in enumerate(generated_answers_dialog):
  print(dialog_questions[i])
  print(ans)
  print()

Кто первым человеком высадился на Луну?
В каком году состоялась эта высадка?
['Первым человеком, высадившимся на Луну, стал Нил Армстронг.', '1969 год.']

Кто написал "Божественную комедию"?
На каком языке написана эта поэма?
['Данте Алигьери', 'Итальянский язык.']

Кто написал роман "1984"?
В каком году был опубликован этот роман?
['Джордж Оруэлл.', '1949 год.']

Кто был премьер-министром Великобритании во время Второй мировой войны?
Сколько лет он находился на посту премьер-министра?
['Уинстон Черчилль', '5 лет.']

Кто исполнил песню "Thriller"?
В каком году был выпущен альбом с этой песней?
['Майкл Джексон', '1982 год.']

Кто создал картину "Мона Лиза"?
В каком музее находится эта картина?
['Леонардо да Винчи.', 'Лувр.']

Кто основал компанию Apple?
В каком году она была основана?
['Основателями компании Apple являются Стив Возняк, Рональд Уэйн и Стив Джобс.', '1976 год']

Кто был президентом США, подписавшим Прокламацию об освобождении рабов?
В каком году это произошло?
['Авраам Ли